# Core Concepts

This chapter covers the fundamental concepts you need to understand before building workflows. Think of this as the "minimum mental model" for n8n.

---

## The Workflow Canvas

A **workflow** is a sequence of connected steps that automate a task.

**Key characteristics:**
- You see nodes on a canvas connected by lines
- Data flows **left-to-right**
- The leftmost node is always a Trigger (what starts the workflow)
- Workflows can be saved as JSON files and shared

---

## Triggers

A **Trigger** is the first node in every workflow. It defines what starts the workflow.

| Trigger Type | What starts the workflow |
|--------------|-------------------------|
| **Manual Trigger** | You click "Execute Workflow" in the editor |
| **Schedule Trigger** | A timer fires (e.g., every hour, daily at 9am) |
| **Webhook Trigger** | An HTTP request from another service |
| **Chat Trigger** | A message in a chat interface |

**For this course:** We use **Manual Trigger** for most examples. This lets you run workflows on demand while learning. Chat Trigger appears later for multi-turn agent conversations.

**Docs:** [Manual Trigger](https://docs.n8n.io/integrations/builtin/core-nodes/n8n-nodes-base.manualworkflowtrigger/)

---

## Nodes

A **Node** is a single step in a workflow.

- Each node performs one action: transform data, call an API, make a decision
- Nodes receive data from the left and send data to the right
- You configure a node by clicking it and editing its parameters

**Node types you will use:**
- **Set / Edit Fields** — create or modify data
- **Basic LLM Chain** — call an AI model
- **Switch** — route data based on conditions
- **Merge** — combine data from multiple branches

---

## Connections

A **Connection** is a line linking two nodes.

| Line Type | What it means |
|-----------|---------------|
| **Solid line** | Data flows from one node to the next |
| **Dotted line** | A capability relationship (e.g., Chat Model → LLM Chain) |

**Data always flows left-to-right.** When you connect two nodes with a solid line, the output of the left node becomes the input of the right node.

**Dotted lines** are special. They indicate that one node provides a capability to another (like a Chat Model providing AI capabilities to an LLM Chain node). No data flows through dotted lines.

---

## Test Mode vs Activated Workflow

There are two ways to run a workflow:

| Mode | What it does | When to use |
|------|-------------|-------------|
| **Test (Execute Workflow)** | Runs once in the editor | While building and debugging |
| **Activated (Toggle ON)** | Runs automatically when triggered | Production use |

**For this course, stay in Test mode.** Click "Execute Workflow" to run your workflow manually.

**Important:** Pinned data (see below) only works in Test mode. Activated workflows always run fresh.

**Docs:** [Workflows](https://docs.n8n.io/workflows/) · [Executions](https://docs.n8n.io/workflows/executions/)

---

## Items: How Data Flows

Data in n8n flows as a list of **items**. Each item is one "thing" being processed.

**Structure of an item:**

| Property | What it contains |
|----------|------------------|
| `.json` | Key-value data (fields like `text`, `topic`, `result`) |
| `.binary` | Binary data (files, images) — less common in AI workflows |

**Example item:**
```json
{
  "topic": "AI in healthcare",
  "audience": "executives"
}
```

When you reference `{{ $json.topic }}`, you are reading the `topic` field from the current item.

---

## The Output Panel

After running a node, click it to see its output in the right panel. There are three views:

| View | Best for |
|------|----------|
| **Table** | Quick reading — shows data in rows and columns |
| **JSON** | Debugging — shows exact data structure with all fields |
| **Schema** | Understanding structure — shows field names and types |

**Recommendation:** Start with **Table** view for readability. Switch to **JSON** when you need to see the exact field names for expressions.

**How to use:**
1. Run the workflow (or a single node)
2. Click any node on the canvas
3. The Output panel appears on the right
4. Toggle between Table, JSON, and Schema views

---

## Pinning Data

**Pinning** saves a node's output so you can reuse it without re-running the node. This is one of the most important features for working efficiently.

**Why pin data:**
- LLM calls cost money and take time
- Pinned data lets you work on downstream nodes without re-running expensive steps
- Debugging is more predictable when data stays constant

**How to pin:**
1. Run the node
2. In the Output panel, click the **pin icon**
3. The node now shows a pin indicator
4. Next time you run the workflow, this node uses pinned data instead of re-executing

**How to unpin:**
- Click the pin icon again
- The node will re-execute next time

**When to pin:**
- After a trigger, so you have stable input data
- After any LLM call you want to keep while testing downstream nodes
- After any slow or costly operation

**Docs:** [Data Pinning](https://docs.n8n.io/data/data-pinning/)

### Common Pitfalls: Pinning

- **Forgetting to unpin:** If data seems stale, check if a node is pinned
- **Pinning at the wrong step:** Pin after the node produces the output you want to keep
- **Pinned data in production:** Pinned data only works in Test mode

---

## Executions Log

The **Executions** panel shows a history of all workflow runs.

**How to access:**
- Click **Executions** in the left sidebar
- Or click the clock icon in the workflow editor

**What you see:**
- Timestamp of each execution
- Status: success or error
- Ability to open and inspect any past execution

**Debugging with Executions:**
1. Find the failed execution in the list
2. Click to open it
3. Click the node with the red error icon
4. Read the error message in the Output panel

**Docs:** [Executions](https://docs.n8n.io/workflows/executions/)

---

## Expressions

**Expressions** let you pass data from one node to another. This is essential for building dynamic workflows.

### Fixed Value vs Expression

Every input field in n8n can be either:

| Mode | What it means |
|------|---------------|
| **Fixed** | Static text that never changes |
| **Expression** | Computed from data in the workflow |

**How to toggle:**
- Click the field
- Look for the toggle that says "Fixed" or "Expression"
- Switch to "Expression" to use dynamic values

### Basic Expression Syntax

Expressions use double curly braces: `{{ }}`

**Read a field from the previous node:**
```
{{ $json.fieldName }}
```

**Read a field from a specific node by name:**
```
{{ $node["Node Name"].json.fieldName }}
```

---

## Expression Examples

### Example 1: Pass data to an LLM prompt

Previous node outputs:
```json
{ "topic": "AI in healthcare" }
```

In the LLM Chain Human Message field:
```
Write a summary about {{ $json.topic }}
```

Result sent to the LLM:
```
Write a summary about AI in healthcare
```

---

### Example 2: Save an LLM response

LLM Chain outputs:
```json
{ "text": "Here is the summary..." }
```

In the next Set node:
- Field name: `summary`
- Value (Expression mode): `{{ $json.text }}`

---

### Example 3: Reference a specific node

You have a node named "Input Data" that set `ticket_body`. Later in the workflow:

```
Analyze this ticket: {{ $node["Input Data"].json.ticket_body }}
```

**Docs:** [Expressions](https://docs.n8n.io/code/expressions/)

---

## Expression Editor Tips

- Type `{{` in any field to see autocomplete suggestions
- The editor shows available fields from previous nodes
- Use the JSON view in the Output panel to find exact field names

### Common Pitfalls: Expressions

- **Forgetting to switch to Expression mode:** Your expression is treated as literal text
- **Typos in field names:** `$json.Text` is not the same as `$json.text` (case-sensitive)
- **Missing quotes around node names:** Use `$node["Node Name"]` with quotes
- **Referencing downstream nodes:** Data flows left-to-right; you cannot reference nodes that have not run yet

---

## Summary: Key Concepts

| Concept | Description |
|---------|-------------|
| **Workflow** | A sequence of connected nodes on a canvas |
| **Trigger** | The first node; defines what starts the workflow |
| **Node** | A single step that performs one action |
| **Connection** | Solid = data flow; Dotted = capability |
| **Item** | One unit of data with `.json` and `.binary` properties |
| **Output Panel** | Where you inspect node results (Table/JSON/Schema) |
| **Pinned Data** | Saved output to reuse without re-running |
| **Execution** | One complete run of a workflow |
| **Expression** | Dynamic value using `{{ $json.field }}` syntax |

---

## Next Steps

Now that you understand the core concepts, proceed to the **Node Toolbox** chapter to learn about the specific nodes used in AI workflows.